## 튜닝용 파라미터

In [126]:
gv_epoch = 4 # 3,2,1           변화 미비
gv_MAX_LEN = 64 
gv_batch_size = 9 #10 ->9  (0.614 10 0.610)
gv_model_nm = 'bert-large-cased' #case -uncase 변화
gv_do_lower_case = False
gv_lr = 4e-6 

gv_num_warmup_steps = 0.2
gv_eps = 1e-8
gv_seed_val = 20200701
gv_random_state =  20200701

In [127]:
!pip install transformers

In [128]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

from google.colab import files
from imblearn.over_sampling import RandomOverSampler

In [129]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [130]:
pd_train = pd.read_json("/content/gdrive/My Drive/Colab Notebooks/friends_train.json") 
pd_test = pd.read_json("/content/gdrive/My Drive/Colab Notebooks/friends_test.json") 
pd_submit = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/en_data.csv",encoding="UTF-8") 

print(pd_train.shape)
print(pd_test.shape)
print(pd_submit.shape)
pd_submit["emotion"] = "neutral"
 

(720, 24)
(200, 24)
(1623, 5)
{'speaker': 'Chandler', 'utterance': 'also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.', 'emotion': 'neutral', 'annotation': '4100000'}
{'speaker': 'Mark', 'utterance': 'Why do all you\x92re coffee mugs have numbers on the bottom?', 'emotion': 'surprise', 'annotation': '2000030'}
   id  i_dialog  ...                                          utterance  emotion
0   0         0  ...                      Alright, whadyou do with him?  neutral
1   1         0  ...                                  Oh! You're awake!  neutral
2   2         0  ...  Then you gotta come clean with Ma! This is not...  neutral
3   3         0  ...                                  Yeah, but this is  neutral
4   4         0  ...          I don't wanna hear it! Now go to my room!  neutral

[5 rows x 6 columns]
        id  ...  emotion
0        0  ...  neutral
1        1  ...  neutral
2        2  ...  neutral
3        3  ...  neutral
4        4  ...  

In [131]:
pd_submit

,id,i_dialog,i_utterance,speaker,utterance,emotion
0,0,0,0,Phoebe,"Alright, whadyou do with him?",neutral
1,1,0,1,Monica,Oh! You're awake!,neutral
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...,neutral
3,3,0,3,Mr. Tribbiani,"Yeah, but this is",neutral
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!,neutral
...,...,...,...,...,...,...
1618,1618,150,14,Joey,Nooo.,neutral
1619,1619,150,15,Lauren,"Hi, Kate!",neutral
1620,1620,150,16,Kate,"Hi, Lauren.",neutral
1621,1621,150,17,Joey,"Hi, Lauren.",neutral


In [132]:
def convert_input_data_flat(p_data):

  x = len(p_data.columns)
  y = len(p_data[0])

  print("convert_input_data_flat(x,y) : " + str(x) + "," +str(y) )  

  pd_xy = pd.DataFrame()

  id = 0 
  i_utterance = 0

  for j in range(y):
    i_utterance = 0
    pd_x = pd.DataFrame()
    for i in range(x):
      #print(i)

      if p_data.loc[j,i] != None:
        pd_x = pd_x.append(pd.DataFrame.from_dict([p_data.loc[j,i]])  )
        id += 1
        i_utterance += 1

    pd_x.insert(0, "i_dialog", j, True)
    pd_x.insert(1, "i_utterance", range(0,i_utterance), True)
    pd_x.drop(['annotation'],axis=1,inplace=True)      
    pd_xy = pd_xy.append(pd_x)

  pd_xy.insert(0, "id", range(0,id), True)    
  pd_xy.index = pd_xy["id"]

  return pd_xy


In [133]:
train = convert_input_data_flat(pd_train)
train["utterance"] = train["speaker"] + " : " + train["utterance"]
train.drop(['speaker'],axis=1,inplace=True)

test = convert_input_data_flat(pd_test)
test["utterance"] = test["speaker"] + " : " + test["utterance"]
test.drop(['speaker'],axis=1,inplace=True)

submit = pd.DataFrame(pd_submit)
submit["utterance"] = submit["speaker"] + " : " + submit["utterance"]
submit.drop(['speaker'],axis=1,inplace=True)
 

convert_input_data_flat(x,y) : 24,720
convert_input_data_flat(x,y) : 24,200
(10561, 5)
(2764, 5)
(1623, 5)


In [134]:
grouped = train.groupby(train['emotion'])
grouped.count()

,id,i_dialog,i_utterance,utterance
emotion,,,,
anger,513,513,513,513
disgust,240,240,240,240
fear,185,185,185,185
joy,1283,1283,1283,1283
neutral,4752,4752,4752,4752
non-neutral,2017,2017,2017,2017
sadness,351,351,351,351
surprise,1220,1220,1220,1220


In [135]:
num_labels = {
    "anger": 0,
    "disgust": 1,
    "fear": 2,
    "joy": 3,
    "neutral": 4,
    "non-neutral": 5,
    "sadness": 6,
    "surprise": 7
}

labels_num = {
    0 : "anger",
    1 : "disgust",
    2 : "fear",
    3 : "joy",
    4 : "neutral",
    5 : "non-neutral",
    6 : "sadness",
    7 : "surprise"
}


In [136]:
train['emotion'] = train['emotion'].replace(num_labels)
test['emotion'] = test['emotion'].replace(num_labels)
submit['emotion'] = submit['emotion'].replace(num_labels)

In [137]:
train.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Chandler : also I was the point person on my c...,4
1,1,0,1,The Interviewer : You mustve had your hands f...,4
2,2,0,2,Chandler : That I did. That I did.,4
3,3,0,3,The Interviewer : So lets talk a little bit a...,4
4,4,0,4,Chandler : My duties? All right.,7


In [138]:
test.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Mark : Why do all youre coffee mugs have numb...,7
1,1,0,1,Rachel : Oh. Thats so Monica can keep track. ...,5
2,2,0,2,Rachel : Y'know what?,4
3,3,0,3,Ross : It didnt.,4
4,4,0,4,"Frank : Okay, so what you used to have with Ra...",3


In [139]:
submit.head()

,id,i_dialog,i_utterance,utterance,emotion
0,0,0,0,"Phoebe : Alright, whadyou do with him?",4
1,1,0,1,Monica : Oh! You're awake!,4
2,2,0,2,Joey : Then you gotta come clean with Ma! This...,4
3,3,0,3,"Mr. Tribbiani : Yeah, but this is",4
4,4,0,4,Joey : I don't wanna hear it! Now go to my room!,4


In [141]:
train.head()

,id,i_dialog,i_utterance,utterance,emotion
id,,,,,
0,0,0,0,Chandler : also I was the point person on my c...,4
1,1,0,1,The Interviewer : You mustve had your hands f...,4
2,2,0,2,Chandler : That I did. That I did.,4
3,3,0,3,The Interviewer : So lets talk a little bit a...,4
4,4,0,4,Chandler : My duties? All right.,7


In [142]:
sentences = train['utterance']
sentences[:10]

id
0    Chandler : also I was the point person on my c...
1    The Interviewer : You mustve had your hands f...
2                   Chandler : That I did. That I did.
3    The Interviewer : So lets talk a little bit a...
4                    Chandler : My duties?  All right.
5    The Interviewer : Now youll be heading a whol...
6                                    Chandler : I see.
7    The Interviewer : But therell be perhaps 30 p...
8                             Chandler : Good to know.
9              The Interviewer : We can go into detail
Name: utterance, dtype: object

In [143]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (train['i_dialog'][i-1] !=  train['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [144]:
sentences[:10]

id
0    [CLS] Chandler : also I was the point person o...
1    [CLS] The Interviewer : You mustve had your h...
2    [CLS] Chandler : That I did. That I did. [SEP]...
3    [CLS] The Interviewer : So lets talk a little...
4    [CLS] Chandler : My duties?  All right. [SEP] ...
5    [CLS] The Interviewer : Now youll be heading ...
6    [CLS] Chandler : I see. [SEP] The Interviewer ...
7    [CLS] The Interviewer : But therell be perhap...
8    [CLS] Chandler : Good to know. [SEP] The Inter...
9    [CLS] The Interviewer : We can go into detail ...
Name: utterance, dtype: object

In [145]:
labels = train["emotion"].values
labels

array([4, 4, 4, ..., 7, 4, 5])

In [146]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print (sentences[0])
print (tokenized_texts[0])

[CLS] Chandler : also I was the point person on my companys transition from the KL-5 to GR-6 system. [SEP] [None] [SEP]
['[CLS]', 'Chandler', ':', 'also', 'I', 'was', 'the', 'point', 'person', 'on', 'my', 'company', '##s', 'transition', 'from', 'the', 'K', '##L', '-', '5', 'to', 'G', '##R', '-', '6', 'system', '.', '[SEP]', '[', 'None', ']', '[SEP]']


In [147]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101, 14394,   131,  1145,   146,  1108,  1103,  1553,  1825,
        1113,  1139,  1419,  1116,  6468,  1121,  1103,   148,  2162,
         118,   126,  1106,   144,  2069,   118,   127,  1449,   119,
         102,   164,  7330,   166,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [148]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [149]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=gv_random_state, 
                                                                                    test_size=0.1)

In [150]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=gv_random_state, 
                                                       test_size=0.1)

In [151]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels) 
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels) 
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  8958,   131,  1573,  1191,  1103, 21406,  1518,  4390,   117,
          146,  1306,  5966,  1214,  1385,  1590,   117,  1150,  1116,  3737,
         1104,  1123, 20559,  2069,   119,   102,  8958,   131,   146,  1400,
         1122,  1121,  1123,  1165,  1131,  1427,  1106,  2631,   117,  4303,
          146,  1180,  1309,  8658,   170,  1282,  1176,  1142,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(4)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([  101, 14394,   131,   146,  1306,  1136,  1256,  2033,  1597,   106,
         3956,   117,  1142,  1110,   170,  2304,  1111,  2598,  4211,  5094,
          106,   102,  9300,   131,  132

In [152]:
batch_size = gv_batch_size

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [153]:
#전처리 - Test Set
sentences = test['utterance']
sentences[:10]

id
0    Mark : Why do all youre coffee mugs have numb...
1    Rachel : Oh. Thats so Monica can keep track. ...
2                                Rachel : Y'know what?
3                                    Ross : It didnt.
4    Frank : Okay, so what you used to have with Ra...
5              Joey : Now, wh-what, what is that like?
6    Frank : Its so cool man, its so, its just ...
7                                   Ross : Yeah, yeah.
8                        Joey : Why cant I find that?
9         Ross : Dont ask me, I had it and I blew it!
Name: utterance, dtype: object

In [154]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (test['i_dialog'][i-1] !=  test['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [155]:
sentences[:10]

id
0    [CLS] Mark : Why do all youre coffee mugs hav...
1    [CLS] Rachel : Oh. Thats so Monica can keep t...
2    [CLS] Rachel : Y'know what? [SEP] Rachel : Oh....
3    [CLS] Ross : It didnt. [SEP] Rachel : Y'know ...
4    [CLS] Frank : Okay, so what you used to have w...
5    [CLS] Joey : Now, wh-what, what is that like? ...
6    [CLS] Frank : Its so cool man, its so, its ...
7    [CLS] Ross : Yeah, yeah. [SEP] Frank : Its so...
8    [CLS] Joey : Why cant I find that? [SEP] Ross...
9    [CLS] Ross : Dont ask me, I had it and I blew...
Name: utterance, dtype: object

In [156]:
labels = test["emotion"].values
labels 

array([7, 5, 4, ..., 4, 4, 4])

In [157]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Mark : Why do all youre coffee mugs have numbers on the bottom? [SEP] [None] [SEP]
['[CLS]', 'Mark', ':', 'Why', 'do', 'all', 'your', '##e', 'coffee', 'mug', '##s', 'have', 'numbers', 'on', 'the', 'bottom', '?', '[SEP]', '[', 'None', ']', '[SEP]']


In [158]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101,  2392,   131,  2009,  1202,  1155,  1240,  1162,  3538,
       15761,  1116,  1138,  2849,  1113,  1103,  3248,   136,   102,
         164,  7330,   166,   102,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [159]:
#어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [160]:
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101,  2392,   131,  2009,  1202,  1155,  1240,  1162,  3538, 15761,
         1116,  1138,  2849,  1113,  1103,  3248,   136,   102,   164,  7330,
          166,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(7)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [161]:
#배치 사이즈
batch_size = gv_batch_size
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [162]:
## 전처리 - Submit Set
submit

,id,i_dialog,i_utterance,utterance,emotion
0,0,0,0,"Phoebe : Alright, whadyou do with him?",4
1,1,0,1,Monica : Oh! You're awake!,4
2,2,0,2,Joey : Then you gotta come clean with Ma! This...,4
3,3,0,3,"Mr. Tribbiani : Yeah, but this is",4
4,4,0,4,Joey : I don't wanna hear it! Now go to my room!,4
...,...,...,...,...,...
1618,1618,150,14,Joey : Nooo.,4
1619,1619,150,15,"Lauren : Hi, Kate!",4
1620,1620,150,16,"Kate : Hi, Lauren.",4
1621,1621,150,17,"Joey : Hi, Lauren.",4


In [163]:
sentences = submit['utterance']
sentences[:10]

0               Phoebe : Alright, whadyou do with him?
1                           Monica : Oh! You're awake!
2    Joey : Then you gotta come clean with Ma! This...
3                    Mr. Tribbiani : Yeah, but this is
4     Joey : I don't wanna hear it! Now go to my room!
5    Chandler : I don?t want him to tell this story...
6    Ross : Oh, but he will. He still tells the sto...
7                          Monica : I wasn?t escaping.
8    Ross : Then how did you get caught in the barb...
9        Monica : I was trying to help out a squirrel.
Name: utterance, dtype: object

In [164]:
sentences_temp = sentences.copy()

for i in range(len(sentences)):
  if (i == 0 ): 
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  elif (submit['i_dialog'][i-1] !=  submit['i_dialog'][i]):  
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] [None] [SEP]" 
  else:    
    sentences_temp[i] = "[CLS] " + str(sentences[i]) + " [SEP] " + str(sentences[i-1]) + " [SEP]"

sentences = sentences_temp

In [165]:
sentences[:10]

0    [CLS] Phoebe : Alright, whadyou do with him? [...
1    [CLS] Monica : Oh! You're awake! [SEP] Phoebe ...
2    [CLS] Joey : Then you gotta come clean with Ma...
3    [CLS] Mr. Tribbiani : Yeah, but this is [SEP] ...
4    [CLS] Joey : I don't wanna hear it! Now go to ...
5    [CLS] Chandler : I don?t want him to tell this...
6    [CLS] Ross : Oh, but he will. He still tells t...
7    [CLS] Monica : I wasn?t escaping. [SEP] Ross :...
8    [CLS] Ross : Then how did you get caught in th...
9    [CLS] Monica : I was trying to help out a squi...
Name: utterance, dtype: object

In [166]:
labels = submit["emotion"].values
labels 

array([4, 4, 4, ..., 4, 4, 4])

In [167]:
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)  
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] Phoebe : Alright, whadyou do with him? [SEP] [None] [SEP]
['[CLS]', 'Phoebe', ':', 'Alright', ',', 'w', '##hady', '##ou', 'do', 'with', 'him', '?', '[SEP]', '[', 'None', ']', '[SEP]']


In [168]:
MAX_LEN = gv_MAX_LEN
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([  101, 19704,   131, 18009,   117,   192, 24905,  6094,  1202,
        1114,  1140,   136,   102,   164,  7330,   166,   102,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [169]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [170]:
submit_inputs = torch.tensor(input_ids)
submit_labels = torch.tensor(labels)
submit_masks = torch.tensor(attention_masks)

print(submit_inputs[0])
print(submit_labels[0])
print(submit_masks[0])

tensor([  101, 19704,   131, 18009,   117,   192, 24905,  6094,  1202,  1114,
         1140,   136,   102,   164,  7330,   166,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(4)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [171]:
batch_size = gv_batch_size
submit_data = TensorDataset(submit_inputs, submit_masks, submit_labels)
submit_sampler = SequentialSampler(submit_data)
submit_dataloader = DataLoader(submit_data, sampler=submit_sampler, batch_size=batch_size, shuffle=False)

In [172]:
device_name = tf.test.gpu_device_name()
#
i#f device_name == '/device:GPU:0':
 #   print('Found GPU at: {}'.format(device_name))
#else:
#    raise SystemError('GPU device not found')


1622

In [173]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [174]:
model = BertForSequenceClassification.from_pretrained(gv_model_nm, num_labels=8)
#model.cuda()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

In [175]:
optimizer = AdamW(model.parameters(), lr = gv_lr, eps = gv_eps )
epochs = gv_epoch
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = gv_num_warmup_steps,
                                            num_training_steps = total_steps)

In [176]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [177]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [178]:
seed_val = gv_seed_val
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model.zero_grad()

In [179]:
t0 = time.time()
total_loss = 0
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [180]:
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    model.to(device) #모델 CUDA 초기화

    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

In [181]:
for epoch_i in range(0, epochs):
    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)           
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))


  예상 Score : 0.607
  예상 Score : 0.616
  예상 Score : 0.631
  예상 Score : 0.621


In [182]:
t0 = time.time()
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

submit_preds =  []

for step, batch in enumerate(submit_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  
    if step < 10:
      print(step)
      print(submit_preds[0:10])
      print(logits.argmax(-1))    
    submit_preds.extend(logits.argmax(-1))     
    if step < 10:
      print(submit_preds[0:10])   
    
print("예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))

예상 Score : 0.556
0
[]
[4 7 5 4 0 6 4 4 4]
[4, 7, 5, 4, 0, 6, 4, 4, 4]
예상 Score : 0.500
1
[4, 7, 5, 4, 0, 6, 4, 4, 4]
[4 0 4 4 4 3 7 6 7]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.444
2
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[5 4 4 4 7 5 3 0 0]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.417
3
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[0 0 0 5 4 6 4 4 5]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.467
4
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[4 4 7 4 4 4 7 5 4]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.500
5
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[7 5 6 4 4 4 4 4 4]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.476
6
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[4 4 7 3 5 5 6 4 5]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.472
7
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[4 4 5 4 7 0 3 7 4]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.444
8
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[4 3 4 0 5 5 5 0 6]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.444
9
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
[7 4 3 4 3 3 4 4 7]
[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]
예상 Score : 0.475
예상 Scor

In [183]:
submit_preds[0:10]

[4, 7, 5, 4, 0, 6, 4, 4, 4, 4]

In [184]:
logits.argmax(-1)

array([4, 4, 3])

In [185]:
print([labels_num[submit_pred] for submit_pred in submit_preds])

['neutral', 'surprise', 'non-neutral', 'neutral', 'anger', 'sadness', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'joy', 'surprise', 'sadness', 'surprise', 'non-neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'non-neutral', 'joy', 'anger', 'anger', 'anger', 'anger', 'anger', 'non-neutral', 'neutral', 'sadness', 'neutral', 'neutral', 'non-neutral', 'neutral', 'neutral', 'surprise', 'neutral', 'neutral', 'neutral', 'surprise', 'non-neutral', 'neutral', 'surprise', 'non-neutral', 'sadness', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'joy', 'non-neutral', 'non-neutral', 'sadness', 'neutral', 'non-neutral', 'neutral', 'neutral', 'non-neutral', 'neutral', 'surprise', 'anger', 'joy', 'surprise', 'neutral', 'neutral', 'joy', 'neutral', 'anger', 'non-neutral', 'non-neutral', 'non-neutral', 'anger', 'sadness', 'surprise', 'neutral', 'joy', 'neutral', 'joy', 'joy', 'neutral', 'neutral', 'surp

In [186]:
t0 = time.time()
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for step, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("예상 Score : {0:.3f}".format(eval_accuracy/nb_eval_steps))

예상 Score : 0.630


In [187]:
submit_pred_labels = [labels_num[submit_pred] for submit_pred in submit_preds]

In [188]:
submission_dic = {"id":list(range(len(submit_pred_labels))), "Predicted":submit_pred_labels} 
submission_df = pd.DataFrame(submission_dic) 
submission_df.to_csv("freinds_nlp_2019511002.csv", index=False)
files.download("freinds_nlp_2019511002.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>